In [1]:
# importamos las librería
# Python version
import sys
print('Python: {}'.format(sys.version))
# scipy
import scipy
print('scipy: {}'.format(scipy.__version__))
# numpy
import numpy
print('numpy: {}'.format(numpy.__version__))
# matplotlib
import matplotlib
print('matplotlib: {}'.format(matplotlib.__version__))

Python: 3.9.5 (default, May 18 2021, 14:42:02) [MSC v.1916 64 bit (AMD64)]
scipy: 1.10.1
numpy: 1.24.3
matplotlib: 3.7.1


In [2]:
# Leer la documentación de SQLALCHEMY
import sqlalchemy

ModuleNotFoundError: No module named 'sqlalchemy'

In [5]:
!pip install psycopg2

In [8]:
!pip install sqlalchemy

     ---------------------------------------- 2.0/2.0 MB 2.9 MB/s eta 0:00:00
     -------------------------------------- 192.1/192.1 kB 3.9 MB/s eta 0:00:00


In [10]:
!pip install seaborn

     -------------------------------------- 293.3/293.3 kB 2.6 MB/s eta 0:00:00


In [6]:
# Leer la documentación de PSYCOPG2
import psycopg2

In [11]:
## DB
from sqlalchemy import create_engine
import psycopg2

## data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd

## visualization
import seaborn as sns
from scipy.stats import norm, skew
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

## Warnings and other tools
import itertools
import warnings
warnings.filterwarnings("ignore")

In [12]:
# Cargamos las credenciales de la base de datos

host = "thebridgechallenge.postgres.database.azure.com"
dbname = "postgres"
user = "postgres@thebridgechallenge"
password = "TheBridge2023"


# Connect to the database
conn = psycopg2.connect(host=host, dbname=dbname, user=user, password=password)

In [ ]:
# Cargamos en pandas la query anterior o consulta de una tabla (TABLE) o cargamos unas vistas (VIEW)

# Obtenemos un dataframe

In [ ]:
# EDA...lo de siempre

In [ ]:
# INSIGHTS

In [ ]:
# Conclusiones

In [ ]:
# Inferir datos con otras fuentes externas con API's

In [ ]:
# Pueden mostrar los datos en Power BI Desktop (free) / Tableau PRO (de pago)

In [ ]:
# Seguir las tareas restantes para paquetizar el trabajo anterior.

# En caso de crear funciones utilizamos bien try...except...finally (pass)
def nombre_funcion():
  try:
    (cond)
    print("Función ejecutada estado 200")
  except Exception as e:
    e
    print("Ha fallado")
  finally:
    print("otra condición final---- ejecutar otra tarea/función")

In [ ]:
# en caso de utilizar logging
import logging

logger = logging.getLogger()
logger.info
logger.warnings
logger.alerts